NAME_1: Marco </br>
LAST-NAME_1: Introvigne </br>
STUDENT-ID_1: 10750466 </br>
NAME_2: Antonio </br>
LAST-NAME_2: Urbano </br>
STUDENT-ID_2: 10527285 </br>
NAME_3: Enrico </br>
LAST-NAME_3: Voltan </br>
STUDENT-ID_3: 10525467 </br>
LEADERBOARD NICKNAME: 3LN (Three Little Neurons) </br>

For each combination of type and company, we define the ‘template’ of the model used during training, we load the saved weights and finally we perform the prediction.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import shutil, random
import time
import matplotlib.pyplot as plt
from tensorflow import keras
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import cm
from tensorflow.keras.applications.vgg16 import preprocess_input 

SEED = 1234
tf.random.set_seed(SEED)
cwd = os.getcwd() 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## VGG Model

In [ ]:
vgg = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(832, 832, 3))
def vggNet(depth, start_f, num_classes):

    model = tf.keras.Sequential()
    
    # Encoder
    model.add(vgg)
    
        
    # Decoder
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))
        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='softmax'))
    
    return model

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
lr = 0
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

def meanIoU(y_true, y_pred):
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3):
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

metrics = ['accuracy', meanIoU]


## UNET

In [ ]:
from drive.MyDrive.Kaggle_2.Unet import build_Unet

## Build Model

In [ ]:
haricot_weedelec_model = build_Unet(input_shape=(768, 768, 3), output_channels=3, depth=5,  base_filters  = 64)
haricot_weedelec_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
haricot_weedelec_model.load_weights("/content/drive/MyDrive/Kaggle_2/HAR_WEED.h5")

haricot_pead_model = build_Unet(input_shape=(768, 768, 3), output_channels=3, depth=5,  base_filters  = 64)
haricot_pead_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
haricot_pead_model.load_weights("/content/drive/MyDrive/Kaggle_2/HAR_PEAD.h5")

mais_weedelec_model = build_Unet(input_shape=(512, 512, 3), output_channels=3, depth=6,  base_filters  = 64)
mais_weedelec_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
mais_weedelec_model.load_weights("/content/drive/MyDrive/Kaggle_2/MAIS_WEED.h5")


In [ ]:
mais_pead_model = build_Unet(input_shape=(512, 512, 3), output_channels=3, depth=5,  base_filters  = 64)
mais_pead_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
mais_pead_model.load_weights("/content/drive/MyDrive/Kaggle_2/MAIS_PEAD.h5")

mais_bipbip_model = build_Unet(input_shape=(512, 512, 3), output_channels=3, depth=4,  base_filters  = 64)
mais_bipbip_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
mais_bipbip_model.load_weights("/content/drive/MyDrive/Kaggle_2/MAIS-BIPBIP.h5")

haricot_bipbip_model = build_Unet(input_shape=(512, 512, 3), output_channels=3, depth=4,  base_filters  = 64)
haricot_bipbip_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
haricot_bipbip_model.load_weights("/content/drive/MyDrive/Kaggle_2/HAR-BIPBIP.h5")

In [ ]:
mais_roseau_model = vggNet(depth=5, 
                     start_f=832, 
                     num_classes=3)
mais_roseau_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
mais_roseau_model.load_weights("/content/drive/MyDrive/Kaggle_2/MAIS-ROS.h5")

haricot_roseau_model = vggNet(depth=5, 
                     start_f=832, 
                     num_classes=3)
haricot_roseau_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
haricot_roseau_model.load_weights("/content/drive/MyDrive/Kaggle_2/HAR_ROS.h5")

## Dictionary

We have created a dictionary in which we join the different results obtained by the single models. 

In [ ]:
input_dict = {}
input_dict["Weedelec"] = {}
input_dict["Weedelec"]["Haricot"] = 768
input_dict["Weedelec"]["Mais"] = 512

input_dict["Bipbip"] = {}
input_dict["Bipbip"]["Haricot"] = 512
input_dict["Bipbip"]["Mais"] = 512

input_dict["Roseau"] = {}
input_dict["Roseau"]["Haricot"] = 832
input_dict["Roseau"]["Mais"] = 832

input_dict["Pead"] = {}
input_dict["Pead"]["Haricot"] = 768
input_dict["Pead"]["Mais"] = 512

model_dict = {}
model_dict["Weedelec"] = {}
model_dict["Weedelec"]["Haricot"] = haricot_weedelec_model
model_dict["Weedelec"]["Mais"] = mais_weedelec_model

model_dict["Bipbip"] = {}
model_dict["Bipbip"]["Haricot"] = haricot_bipbip_model
model_dict["Bipbip"]["Mais"] = mais_bipbip_model

model_dict["Roseau"] = {}
model_dict["Roseau"]["Haricot"] = haricot_roseau_model
model_dict["Roseau"]["Mais"] = mais_roseau_model

model_dict["Pead"] = {}
model_dict["Pead"]["Haricot"] = haricot_pead_model
model_dict["Pead"]["Mais"] = mais_pead_model

split_dict = {}
split_dict["Weedelec"] = {}
split_dict["Weedelec"]["Haricot"] ={}
split_dict["Weedelec"]["Haricot"]["cols"] = 2
split_dict["Weedelec"]["Haricot"]["rows"] = 2
split_dict["Weedelec"]["Mais"] ={}
split_dict["Weedelec"]["Mais"]["cols"] = 2
split_dict["Weedelec"]["Mais"]["rows"] = 2

split_dict["Bipbip"] = {}
split_dict["Bipbip"]["Haricot"] ={}
split_dict["Bipbip"]["Haricot"]["cols"] = 2
split_dict["Bipbip"]["Haricot"]["rows"] = 2
split_dict["Bipbip"]["Mais"] ={}
split_dict["Bipbip"]["Mais"]["cols"] = 2
split_dict["Bipbip"]["Mais"]["rows"] = 2

split_dict["Roseau"] = {}
split_dict["Roseau"]["Haricot"] ={}
split_dict["Roseau"]["Haricot"]["cols"] = 2
split_dict["Roseau"]["Haricot"]["rows"] = 1
split_dict["Roseau"]["Mais"] ={}
split_dict["Roseau"]["Mais"]["cols"] = 2
split_dict["Roseau"]["Mais"]["rows"] = 1

split_dict["Pead"] = {}
split_dict["Pead"]["Haricot"] ={}
split_dict["Pead"]["Haricot"]["cols"] = 2
split_dict["Pead"]["Haricot"]["rows"] = 2
split_dict["Pead"]["Mais"] ={}
split_dict["Pead"]["Mais"]["cols"] = 2
split_dict["Pead"]["Mais"]["rows"] = 2



In [ ]:
def read_names(images_path):
    """ Find all the images from the images_path directory and
        the segmentation images from the segs_path directory
        while checking integrity of data """

    image_files = []
    print(images_path)
    for dir_entry in os.listdir(images_path):
        if os.path.isfile(os.path.join(images_path, dir_entry)):
            image_files.append(dir_entry)

    return image_files

In [ ]:
import math
def cropImg(im, rows, cols): 
  width = math.ceil(im.size[0]/cols)
  height = math.ceil(im.size[1]/rows)
  parts = []
  for i in range(rows):
    for j in range(cols):
      right_w = (j+1)*width if j < cols - 1 else im.size[0]
      right_h = (i+1)*height if i < rows - 1 else im.size[1]
      box = (j*width, i*height, right_w, right_h)
      parts.append(im.crop(box))
  return parts

In [ ]:
class CustomTestDataset(tf.keras.utils.Sequence):

  def __init__(self, dataset_dir, out_shape=(800,800), cols = 2, rows = 2, preprocess = None):
        
    subset_filenames = read_names(dataset_dir)   
    print(len(subset_filenames))
    self.dataset_dir = dataset_dir    
    self.subset_filenames = subset_filenames
    self.out_shape = out_shape
    self.cols = cols
    self.rows = rows
    self.pre = preprocess
  

  def __len__(self):
    return len(self.subset_filenames)* self.cols * self.rows

  def __getitem__(self, index):
    curr_filename = self.subset_filenames[index // (self.cols * self.rows)]
    img = Image.open(os.path.join(self.dataset_dir, curr_filename))
    x = index % (self.cols * self.rows)
    img = cropImg(img,self.rows,self.cols)[x]
    img = img.resize(self.out_shape) 
    img_arr = np.array(img)
    if self.pre is not None:
      img_arr = self.pre(img_arr)

    return img_arr.astype(float)/255, np.array([curr_filename])

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
colors = []
colors.append([0, 0, 0]) # background
colors.append([255, 255, 255] ) # crop
colors.append([216, 67, 82])  # weed

In [ ]:
n_classes = 3

Two methods used to join the cropped predictions in order to obtain the original one

In [ ]:
def join_hor(im1, im2):
   imgs = [ im1, im2 ]
   min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
   imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
   return Image.fromarray(imgs_comb)

def join_vert(im1, im2):
   imgs = [ im1, im2 ]
   min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
   imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
   return Image.fromarray(imgs_comb)

## Create .json

In [ ]:
import json
from tensorflow.keras.applications.vgg19 import preprocess_input 

aziende = ["Bipbip", "Pead", "Roseau", "Weedelec"]
tipi = ["Mais","Haricot"]
submission_dict = {}
for azienda in aziende:
  for tipo in tipi:
    input_height = input_dict[azienda][tipo]
    output_height = input_dict[azienda][tipo]
    model = model_dict[azienda][tipo]
    cols = split_dict[azienda][tipo]["cols"]
    rows = split_dict[azienda][tipo]["rows"]
    preprocess = preprocess_input if azienda == "Roseau" else None # and tipo == "Haricot"


    test_path = f"/content/drive/MyDrive/Kaggle_2/Development_Dataset/Test_Dev/{azienda}/{tipo}/Images"
    ds = CustomTestDataset(test_path,(input_height,input_height), cols, rows, preprocess)
    test_dataset = tf.data.Dataset.from_generator(lambda: ds,
                                               output_types=(tf.float32, tf.string),
                                               output_shapes=([input_height, input_height, 3],[1,]))

    test_dataset = test_dataset.repeat()

    iterator = iter(test_dataset)
    for t in range(15):      
      parts = []
      for i in range(rows):
        for j in range(cols):
          image, img_name = next(iterator)
          #out_sigmoid = model.predict(x=tf.expand_dims(image, 0)) -> prediction without computing any rotation
          
          # We perform 4 rotation of 90° to all the cropped part
          out_sigmoid_0 = model.predict(x=tf.expand_dims(image, 0))
          out_sigmoid_1 = model.predict(x=tf.expand_dims(tf.image.rot90(image,1), 0))
          out_sigmoid_2 = model.predict(x=tf.expand_dims(tf.image.rot90(image,2), 0))
          out_sigmoid_3 = model.predict(x=tf.expand_dims(tf.image.rot90(image,3), 0))

          # Sum of the prediction
          out_sigmoid = tf.image.rot90(out_sigmoid_3,1) + tf.image.rot90(out_sigmoid_2,2) + tf.image.rot90(out_sigmoid_1,3) + out_sigmoid_0
          predicted_class = tf.argmax(out_sigmoid, -1)
          predicted_class = predicted_class[0, ...]
          prediction_img = np.zeros([predicted_class.shape[0], predicted_class.shape[1], 3])
          prediction_img[np.where(predicted_class == 0)] = [0, 0, 0]
          for i in range(1, 3):
            prediction_img[np.where(predicted_class == i)] = np.array(colors[i])
          png = Image.fromarray(np.uint8(prediction_img),"RGB")
          parts.append(png)
      print(f"collected {len(parts)}")
      
      # Here we join all the cropped predictions
      finalPrediction = None
      for r in range(rows):
        c = 1
        r_img = parts[r * cols]
        while c < cols:
          r_img = join_hor(r_img, parts[r * cols + c])
          c +=1
        if r == 0:
          finalPrediction = r_img
        else:
          finalPrediction = join_vert(finalPrediction, r_img)
        
      print(finalPrediction.size)

      
      name = tf.compat.as_str_any(img_name).split("'")[1].split("'")[0]
      sizes = Image.open(test_path + "/" + name).size

      print(sizes)
      pred_resize = finalPrediction.resize(sizes, resample=Image.NEAREST)

      img_array = np.array(pred_resize)
      new_mask_arr = np.zeros(sizes[::-1] , dtype=img_array.dtype)
      new_mask_arr[np.where(np.all(img_array == [255, 255, 255], axis=-1))] = 1
      new_mask_arr[np.where(np.all(img_array == [216, 67, 82], axis=-1))] = 2
      
      mask_arr = new_mask_arr
      name = name[:-4]

      print(f"predicted {name}")
      submission_dict[name] = {}
      submission_dict[name]['shape'] = mask_arr.shape
      submission_dict[name]['team'] = azienda
      submission_dict[name]['crop'] = tipo
      submission_dict[name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[name]['segmentation']['weed'] = rle_encoded_weed

with open(f'./submission.json', 'w') as f:
    json.dump(submission_dict, f)
    

